# Import needed library

In [ ]:
import random
import sqlite3

# Define function

## SQL database

In [8]:
def create_database():
    try:
        conn = sqlite3.connect('card.s3db')
        cur = conn.cursor()
        cur.execute('''CREATE TABLE card(id INTEGER, number TEXT, pin TEXT, balance INTEGER DEFAULT 0)''')
        print('Table Created.')
        conn.commit()
        conn.close()
    except sqlite3.Error as error:
        print(error)


def insert_value(ccard, pw):
    try:
        conn = sqlite3.connect('card.s3db')
        cur = conn.cursor()
        cur.execute('''INSERT INTO card (number, pin) VALUES (?, ?)''',
                    (ccard, pw,))
        conn.commit()
        conn.close()
    except sqlite3.Error as error:
        print(error)


def load_value():
    try:
        conn = sqlite3.connect('card.s3db')
        cur = conn.cursor()
        cur.execute('''SELECT * FROM card''')
        records = cur.fetchall()
        conn.close()
        credit_card_list = [records[i][1] for i in range(len(records))]
        passwords_list = [records[i][2] for i in range(len(records))]
        balance_list = [records[i][3] for i in range(len(records))]
        return credit_card_list, passwords_list, balance_list
    except sqlite3.Error as error:
        print(error)


def update_value(credit_card, entered_balance):
    try:
        conn = sqlite3.connect('card.s3db')
        cur = conn.cursor()
        cur.execute('''UPDATE card SET balance = (?) WHERE number = (?)''', (entered_balance, credit_card))
        conn.commit()
        conn.close()
    except sqlite3.Error as error:
        print(error)


def delete_account(ccard):
    try:
        conn = sqlite3.connect('card.s3db')
        cur = conn.cursor()
        cur.execute('''DELETE FROM card WHERE number = (?)''', (ccard,))
        conn.commit()
        conn.close()
    except sqlite3.Error as error:
        print(error)


def main_choices():
    choice = input('1. Create an account\n2. Log into account\n0. Exit\n')
    return choice


def create_choice():
    new_card = create_creditcard()
    new_pw = str(random.randint(0, 9999)).zfill(4)
    print('Your card has been created')
    print(f'Your card number:\n{new_card}')
    print(f'Your card PIN:\n{new_pw}')
    return new_card, new_pw


def create_creditcard():
    new_card = list("400000" + str(random.randint(0, 999999999)).zfill(9))
    new_card = list(map(int, new_card))
    temp_card = []
    for i in list(range(0, 15)):
        x = new_card[i]
        if i % 2 == 0:
            temp_card.append(x * 2 - 9 if x * 2 > 9 else x * 2)
        else:
            temp_card.append(x)
    new_card.append((10 - sum(temp_card) % 10) % 10)
    new_card = ''.join(list(map(str, new_card)))
    return new_card


def login_choice(credit_card_list, password_list):
    chk_card = input('Enter your card number:\n')
    chk_pin = input('Enter your PIN:\n')
    if chk_card not in credit_card_list or (chk_card in credit_card_list and password_list[credit_card_list.index(chk_card)] != chk_pin):
        print('Wrong card number or PIN!')
        return False
    else:
        print('You have successfully logged in!')
        return chk_card


def after_login():
    choice = input('1. Balance\n2. Add income\n3. Do transfer\n4. Close account\n5. Log out\n0. Exit\n')
    return choice


def after_login2(credit_card_list, balance_list):
    add_balance = int(input('Enter income:\n'))
    updated_balance = balance_list[credit_card_list.index(user_cc)] + add_balance
    return updated_balance


def check_alg(credit_card):
    card_list = list(credit_card)
    card_list = [int(card_list[i])*2 if i % 2 == 0 else int(card_list[i]) for i in range(16)]
    card_list = [i - 9 if i >= 10 else i for i in card_list]
    if sum(card_list) % 10 == 0:
        return True
    else:
        return False


def transfer_balance(credit_card_list, balance_list, user_card, trans_card, transfer_money):
    bl_1 = balance_list[credit_card_list.index(user_card)] - transfer_money
    bl_2 = balance_list[credit_card_list.index(trans_card)] + transfer_money
    return bl_1, bl_2

## System running

In [9]:
create_database()  # creating the database if the database doesn't exist
sys = True
while sys:
    user = main_choices()  # user input for creating account or logging in
    if user == '1':
        card, pin = create_choice()  # auto generate new account with pin
        insert_value(card, pin,)
    elif user == '2':
        ccs, pws, bls = load_value()  # read database
        user_cc = login_choice(ccs, pws)  # checking account number and pin match
        if not user_cc:
            continue
        else:
            login_sys = True  # log in success
            while login_sys:
                # user input for log in menu
                # 1 => checking balance
                # 2 => adding balance
                # 3 => transfer balance
                # 4 => closing account
                # 5 =>  log out
                user_login = after_login()
                if user_login == '1':  # checking balance
                    balance = bls[ccs.index(user_cc)]
                    print(f'Balance: {balance}')
                elif user_login == '2':  # adding balance
                    new_balance = after_login2(ccs, bls)
                    update_value(user_cc, new_balance)
                    bls = load_value()[2]
                elif user_login == '3':  # transfer balance
                    trans_to = input('Transfer\nEnter card number:\n')
                    if trans_to == user_cc:
                        print("You can't transfer money to the same account!")
                    elif not check_alg(trans_to):  # check if the transfer account satisfied Luhn algorithm
                        print("Probably you made a mistake in the card number. Please try again!")
                    elif check_alg(trans_to) and trans_to not in ccs:  # satisfied Luhn algorithm but not in data base
                        print("Such a card does not exist.")
                    else:  # transfer account exist
                        transfer_amount = int(input("Enter how much money you want to transfer:\n"))
                        if transfer_amount > bls[ccs.index(user_cc)]:
                            print("Not enough money!")
                        else:
                            print("Success!")
                            new_balance1, new_balance2 = transfer_balance(ccs, bls, user_cc, trans_to, transfer_amount)
                            update_value(user_cc, new_balance1)
                            update_value(trans_to, new_balance2)
                            bls = load_value()[2]
                elif user_login == '4':  # closing account
                    delete_account(user_cc)
                    print("The account has been closed!")
                    login_sys = False
                elif user_login == '5':  # log out
                    print('You have successfully logged out!')
                    break
                elif user_login == '0':  # exit system
                    print('Bye!')
                    login_sys, sys = False, False
                else:
                    print('Invalid input')
    elif user == "0":  # exit system
        sys = False
        print('Bye!')
    else:
        print('Invalid input')

table card already exists


1. Create an account
2. Log into account
0. Exit
 1


Your card has been created
Your card number:
4000002757739261
Your card PIN:
9184


1. Create an account
2. Log into account
0. Exit
 1


Your card has been created
Your card number:
4000006376815170
Your card PIN:
2696


...

The account has been closed!


1. Create an account
2. Log into account
0. Exit
 2
Enter your card number:
 4000006376815170
Enter your PIN:
 2696


Wrong card number or PIN!


1. Create an account
2. Log into account
0. Exit
 0


Bye!
